In [ ]:
# Notebook to adjust state-level solar prices and calculate appropriate price decline for baseline scenario

In [ ]:
import os, sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

sys.path.append(os.path.abspath(".."))

from input_data_functions import stacked_sectors

In [ ]:
# Load large file
lbnl = pd.read_csv('../../../data/TTS_LBNL_public_file_29-Sep-2025_all.csv')

In [ ]:
# Load data
pv = pd.read_csv('../../input_data/pv_prices/pv_price_atb23_mid.csv')
batt = pd.read_csv('../../input_data/batt_prices/batt_prices_FY23_mid.csv')
pv_batt = pd.read_csv('../../input_data/pv_plus_batt_prices/pv_plus_batt_prices_FY23_mid.csv') 
inflation = pd.read_csv('../../../data/inflation_multipliers.csv')

In [ ]:
# Assign state
STATE = 'VA'

# First filter LBNL data for relevant technology and customer segment
lbnl_filt = (
    lbnl[
        (lbnl['technology_type'].isin(['pv-only'])) &
        (lbnl['customer_segment'].isin(['RES', 'RES_SF'])) &
        (lbnl['total_installed_price'] > 0) &
        (
            (lbnl['PV_system_size_DC'] > 0)
        )
    ]
).copy()

# If the necessary state exists, filter for that state. If not, take whole US
if STATE in lbnl_filt['state'].unique():
    lbnl_filt = lbnl_filt[lbnl_filt['state'] == STATE] 
else:
    print(f"{STATE} not found in LBNL data, using all US data instead.")


# First extract year month day from the date column
lbnl_filt['parsed_date'] = pd.to_datetime(lbnl_filt['installation_date'], format="%Y-%m-%d", errors='coerce')

# Extract year, month, day into new columns
lbnl_filt['year'] = lbnl_filt['parsed_date'].dt.year
lbnl_filt['month'] = lbnl_filt['parsed_date'].dt.month
lbnl_filt['day'] = lbnl_filt['parsed_date'].dt.day

# 2024 prices
lbnl_filt = lbnl_filt[lbnl_filt['year'] >= 2019]

# Join to inflation data
lbnl_inflation = lbnl_filt.merge(inflation, on=['year', 'month'], how='left')

# Inflation adjust prices
lbnl_inflation['total_installed_price'] = lbnl_inflation['total_installed_price'] * lbnl_inflation['inflation_multiplier']  
lbnl_inflation['battery_price'] = lbnl_inflation['battery_price'] * lbnl_inflation['inflation_multiplier']  

# Calculate median solar and prices over the last four years

# PV prices when the tech is either 'pv-only' or 'pv+storage', use the total_installed_price
price_per_kw_solar = (
    lbnl_inflation[
        (lbnl_inflation['technology_type'].isin(['pv-only', 'pv+storage'])) &
        (~lbnl_inflation['PV_system_size_DC'].isna()) &
        (~lbnl_inflation['total_installed_price'].isna())
        ][['PV_system_size_DC', 'total_installed_price', 'state', 'year']]
)
price_per_kw_solar['price_per_kw'] = price_per_kw_solar['total_installed_price'] / price_per_kw_solar['PV_system_size_DC']

In [ ]:
# 1. compute median price per year
medians = (
    price_per_kw_solar.groupby('year', as_index=False)['price_per_kw']
      .median()
      .sort_values('year')
)

# 2. compute year-over-year % change
medians['pct_change'] = medians['price_per_kw'].pct_change()

# 3. compute average *decrease* (%)
avg_pct_decrease = -medians['pct_change'].mean() * 100  # negative → decrease

print(f"Average yearly decrease: {avg_pct_decrease:.2f}%")

In [ ]:
pv_batt = stacked_sectors(pd.read_csv(f'../../input_data/pv_plus_batt_prices/pv_plus_batt_prices_FY23_{STATE}_baseline.csv'))
pv_batt_dollar_per_watt = stacked_sectors(pd.read_csv(f'../../input_data/pv_plus_batt_prices/pv_plus_batt_prices_FY23_{STATE}_policy.csv'))

In [ ]:
# Upload new DFs to Cloud SQL Instance

# Connection config
DB_USER = "postgres"
DB_PASS = "postgres"
DB_NAME = "dgendb"
DB_PORT = 5432
DB_HOST = "127.0.0.1"  # local Cloud SQL Proxy

# Build SQLAlchemy connection string
conn_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create engine
engine = create_engine(conn_str)

# Replace the relevant tables in the database
pv_batt.to_sql("pv_plus_batt_baseline", engine, schema="diffusion_shared",  if_exists="replace", index=False)

# Replace the relevant tables in the database
pv_batt_dollar_per_watt.to_sql("pv_plus_batt_dollar_per_watt", engine, schema="diffusion_shared",  if_exists="replace", index=False)

engine.dispose()